The Turán number of $C_5^-$
==============================

This notebook contains calculations for the Turán number of $C_5^-$

As the blowup of $K_4^-$ contains $C_5^-$, we can additionally assume 
that we work in $K_4^-$-free structures. 

1. The first cell sets up the combinatorial theory of $C_5^-$ and 
$K_4^-$-free 3-graphs (called TGp). In addition, it sets up the 
combinatorial theory on the same 3-graphs with vertices colored 
from 3 possible colors (called CTGp)

2. The second cell performs the basic calculation of upper bounding
edges in the theory. It gives the 1/4 + 1/1000 (in fact 0.250728)
upper bound. The certificate proving the claim is in the file "noc5m_0.json".

3. That upper bound is used in the following cell. It lower bounds the
quotient k222/k111 near the optimum (where this value is 24/121~0.198),
by around 0.194. The certificate proving the claim is in the file "noc5m_1a.json".

4. The next cell minimizes k222 (which is easier to argue with in the
paper) near the optimum. At the optimum, this value is 6/121~0.0495 and
the code provides the bound 0.0487 . The certificate proving the claim is in
the file "noc5m_1b.json".

5. The last cell works in the colored theory, and shows that the bad
edges have number smaller than the missing good edges. It uses the upper
bound about k222/k111. The calculations provide the precise density bound
that there are less bad edges than missing edges asymptotically.
(Even stability holds). The certificate proving this claim is in the file "noc5m_2.json".

In [1]:
### This cell is just to set up the theory.
### In practice it is not needed, as the calculations (multiplication table and generated structures)
### are already done and saved. But it is here for completeness (and for re-runs from scratch)

from sage.algebras.flag_algebras import *

# These are helper functions, to deal with classical exclusion (not just induced)
def check_containment(smalls, larges):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
TG.exclude()

# C5 minus
C5m = TG(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])

# flags of size 5
fl5 = TG.generate_flags(5)

# boolean vector indicating each element in fl5 if it has C5m
gs = check_containment([C5m], fl5)

# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])

# set the excluded structures. k4, k4m and all in fl5 containing C5m
exls = [k4, k4m] + [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partt = list(ftype_points) + \
            [[ii for ii in range(n) if ii not in ftype_union]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoC5m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))




# This is code to create colored theories
# This is a default code for all color partition
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    ftype_union = [jj for ff in ftype_points for jj in ff]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = list(ftype_points) + \
              [[ii for ii in range(n) if ii not in ftype_union]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))
    
    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# This is also a default code for all color partition
def _generateCT(base_theory, k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    BT = base_theory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

# To make the default codes work for this specific case:
# The generator:
# Colors the elements of TGp (3-graphs without C5- and K4-), works on 3-uniform structures
# and the colors 0, 1, 2 are interchangeable (otherwise it would say [[0], [1], [2]]
def generate_colored(n):
    return _generateCT(TGp, 3, [[0, 1, 2]], n)

# Same for the identifier. Colors are interchangeable.
def identify_colored(n, ftype_points, edges, C0, C1, C2):
    return _identifyCT(3, [[0, 1, 2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

# CTGp is the colored variant of TGp (NoC5m)
CTGp = CombinatorialTheory("ColoredNoC5m", generate_colored, identify_colored, edges=3, C0=1, C1=1, C2=1)

# sanity check, the number of flags with size 4, 5, 6
# should be 18 132 2840
print(len(CTGp.generate_flags(4)), len(CTGp.generate_flags(5)), len(CTGp.generate_flags(6)))

9 55 1127
18 132 2840


In [4]:
### This is where the actual calculation starts.
### This part just gives a standard upper bound on the number of edges

# degree equality
p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]

pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])

standard_assums = [degree_difference, -degree_difference, pointed_edge-1/4]

#run the optimizer
max_edge = TGp.optimize(TGp(3, edges=[[0, 1, 2]]), 7, positives=standard_assums, exact=True, denom=1024*1024, file="no_c5m_0")
print(max_edge['result'].n()) #should be a little larger than 1/4 = 0.25

0.250728940206861


In [5]:
### This code block deals with the uncolored part of the calculation (minimizing k222/k111)

p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]
pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])


import itertools
# base edges for f222 (0, 1, 2) is the center and (3, 4, 5) is the good edge
be = [[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]
# additional edges for f222. These are the edges we can include while still not violating the no C5m condition.
ae = [[0, 4, 5], [1, 3, 5], [2, 3, 4]]

# f222, it is a sum of 8 flags, so simply use this iterator to loop through all subsets of the additional edges
f222 = TGp(6, ftype=[0, 1, 2], edges=be)
for ii in [1, 2, 3]:
    for xx in itertools.combinations(ae, int(ii)):
        ad = TGp(6, ftype=[0, 1, 2], edges=be+list(xx))
        f222 = f222 + ad

# standard positivity constraints, the edge <= 1/4 + 1/1000 is used here from the previous cell
edge_lower = TGp(3, edges=[[0, 1, 2]]) - 1/4
edge_upper = 1/4 + 1/1000 - TGp(3, edges=[[0, 1, 2]])

# the list of positivity assumptions
uncolored_assums = [edge_lower, edge_upper, degree_difference, -degree_difference]

# This is for minimizing f222, which is a flag, so technically it is the quotient k222 / k111
min_f222 = TGp.optimize(f222, 7, maximize=False, positives=uncolored_assums, exact=True, denom=1024*1024, file="no_c5m_1a")
print(min_f222['result'].n()) #should be a little smaller than 24/121 ~ 0.19834710743

Base flags generated, their number is 1127
The relevant ftypes are constructed, their number is 12
Block sizes before symmetric/asymmetric change is applied: [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28]


Done with mult table for Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]]: : 12it [00:00, 911.24it/s]


Tables finished


Done with positivity constraint 3: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]


Constraints finished
Running sdp without construction. Used block sizes are [5, 21, 53, 11, 21, 14, 374, 31, 147, 35, 48, 23, 74, 10, 24, 28, 17, 23, 31, 12, 19, 8, 20, -1127, -102]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.36e-01 Pobj: -7.6377951e+01 Ad: 4.01e-02 Dobj:  7.6684343e-01 
Iter:  2 Ap: 6.42e-01 Pobj: -4.3747042e+02 Ad: 2.09e-01 Dobj:  2.5738466e+00 
Iter:  3 Ap: 6.81e-01 Pobj: -7.9572498e+02 Ad: 5.61e-01 Dobj:  2.3001482e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -1.2710643e+03 Ad: 7.02e-01 Dobj:  6.7484539e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.2444092e+03 Ad: 9.05e-01 Dobj:  1.5908011e-01 
Iter:  6 Ap: 8.99e-01 Pobj: -1.0740863e+03 Ad: 8.25e-01 Dobj:  1.0390679e-01 
Iter:  7 Ap: 4.14e-02 Pobj: -1.0368739e+03 Ad: 2.88e-01 Dobj:  1.3461514e-01 
Iter:  8 Ap: 2.05e-01 Pobj: -9.6481959e+02 Ad: 4.05e-01 Dobj:  1.2273798e-01 
Iter:  9 Ap: 6.37e-01 Pobj: -5.2152294e+02 Ad: 4.96e-01 Dobj:  1.4648169e-01 
Iter: 10 Ap: 9.18e-01 Pobj:

100%|███████████████████████████████████████████| 23/23 [00:14<00:00,  1.59it/s]


Calculating resulting bound


100%|███████████████████████████████████████████| 12/12 [08:33<00:00, 42.81s/it]


Rounding errors are [9.458030241051693e-07, 4.146376346038885e-06, 6.474227887520624e-06, 2.141028596767542e-06, 3.4345168457969868e-06, 3.7035286113779943e-06, 2.121547149819364e-05, 3.712461202962111e-06, 7.362130826233936e-06, 4.075160989083135e-06, 3.5645890169504884e-06, 2.6735498666103573e-06, 4.621737661163215e-06, 2.16275463858672e-06, 3.2004617372597644e-06, 2.681818046697396e-06, 1.938213383783338e-06, 2.4408287748325868e-06, 2.7581126550855827e-06, 3.4838723903816335e-06, 2.5349875323671506e-06, 1.9193507014141106e-06, 2.9287973504665624e-06, 1.9601398519015877e-06]
0.194809940126207


In [6]:
### This alternative avoids the calculation with the quotient k222/k111, instead minimizes k222

# these are the projected values. Here the automorphism groups are taken care of.
k222 = f222.project()
min_k222 = TGp.optimize(k222, 7, maximize=False, positives=uncolored_assums, exact=True, denom=1024*1024, file="no_c5m_1b")
print(min_k222['result'].n()) #should be a little smaller than 6/121 ~ 0.04958677685

Base flags generated, their number is 1127
The relevant ftypes are constructed, their number is 12
Block sizes before symmetric/asymmetric change is applied: [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28]


Done with mult table for Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]]: : 12it [00:00, 901.02it/s]


Tables finished


Done with positivity constraint 3: 100%|██████████| 4/4 [00:02<00:00,  1.49it/s]


Constraints finished
Running sdp without construction. Used block sizes are [5, 21, 53, 11, 21, 14, 374, 31, 147, 35, 48, 23, 74, 10, 24, 28, 17, 23, 31, 12, 19, 8, 20, -1127, -102]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 5.84e-01 Pobj: -8.6912595e+01 Ad: 1.51e-01 Dobj:  2.6900305e+00 
Iter:  2 Ap: 7.34e-01 Pobj: -2.0473536e+02 Ad: 5.13e-01 Dobj:  2.4358993e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -3.6118024e+02 Ad: 7.40e-01 Dobj:  5.5090242e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -3.6071706e+02 Ad: 9.06e-01 Dobj:  9.0755278e-02 
Iter:  5 Ap: 1.00e+00 Pobj: -2.8977107e+02 Ad: 8.97e-01 Dobj:  3.2558573e-02 
Iter:  6 Ap: 6.48e-01 Pobj: -2.5680903e+02 Ad: 6.49e-01 Dobj:  2.6871477e-02 
Iter:  7 Ap: 5.36e-01 Pobj: -2.1059796e+02 Ad: 3.71e-01 Dobj:  3.0155148e-02 
Iter:  8 Ap: 1.00e+00 Pobj: -1.4879854e+02 Ad: 6.12e-01 Dobj:  3.5919566e-02 
Iter:  9 Ap: 1.00e+00 Pobj: -1.1543730e+02 Ad: 5.92e-01 Dobj:  4.1257446e-02 
Iter: 10 Ap: 6.99e-01 Pobj:

100%|███████████████████████████████████████████| 23/23 [00:09<00:00,  2.49it/s]


Calculating resulting bound


100%|███████████████████████████████████████████| 12/12 [06:18<00:00, 31.58s/it]


Rounding errors are [1.926102739225377e-06, 4.251280493292014e-06, 6.336425660467133e-06, 2.3715273616781512e-06, 3.5747462837776344e-06, 2.812407594638239e-06, 1.9806837562520336e-05, 3.5970164358046335e-06, 7.495772522089308e-06, 3.7439267822169474e-06, 3.426566221626672e-06, 2.496405295893567e-06, 4.774162795895738e-06, 2.4954896136196245e-06, 3.3075583333466476e-06, 2.7544249200314487e-06, 1.9342072184340427e-06, 2.6379804478424042e-06, 3.025562575891906e-06, 2.8096946192096314e-06, 3.041006075167943e-06, 1.9290241189709353e-06, 2.780990504464593e-06, 2.9248118191629557e-06]
0.0487234266977462


In [7]:
### This is the code that performs the calculations on the colored theory

#edge with (C)orrect colors
C = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
#edge with (C)orrect colors (p)ointed
Cp = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], ftype=[0])

#edge with (B)ad colors (since color-blind, this includes the bad edges looking the wrong way too)
B = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])

#edge with (B)ad colors (p)ointed 
Bp = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [2]], C1=[[1]], C2=[], ftype=[0])

#(M)issing edge with good colors
M = CTGp(3, edges=[], C0=[[0]], C1=[[1]], C2=[[2]])

#positivity assumptions
#each point, good edges are more than bad edges (divided by two due to the wrong color order)
#edge density is larger than 0.194 (from previous calculation)
colored_assums = [Cp - Bp/2, C - 194/1000]

#optimal construction and its derivatives
optim = CTGp.blowup_construction(6, 3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], symmetric=True, symbolic=True)
ssoptim = optim.set_sum()
der_optims = ssoptim.derivatives([1/3, 1/3])

#bad is less than missing, proven by (B)ad (M)inum (M)issing is at most 0.
max_bmm = CTGp.optimize(B + (-99/100)*M, 6, maximize=True, positives=colored_assums, 
                        exact=True, construction=der_optims, denom=2048, file="no_c5m_2")
print(max_bmm['result']) #should be exactly 0

100%|███████████████████████████████████████████| 28/28 [00:01<00:00, 22.08it/s]


Base flags generated, their number is 2840
The relevant ftypes are constructed, their number is 20
Block sizes before symmetric/asymmetric change is applied: [35, 57, 82, 123, 123, 123, 40, 60, 60, 60, 60, 60, 20, 30, 30, 30, 30, 30, 30, 30]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]], C0=[[0]], C1=[[1]], C2=[[2], [3]]: : 20it [00:01, 11.59it/s]    


Tables finished


Done with positivity constraint 1: 100%|██████████| 2/2 [00:02<00:00,  1.31s/it]


Constraints finished
Adjusting table with kernels from construction
Running SDP after kernel correction. Used block sizes are [23, 9, 29, 24, 12, 68, 33, 87, 29, 91, 43, 80, 14, 26, 21, 39, 39, 21, 39, 21, 39, 21, 33, 27, 12, 8, 21, 9, 18, 12, 13, 17, 24, 6, 17, 13, 30, 12, 15, -2840, -50]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 3.71e-01 Pobj: -3.3411032e+01 Ad: 1.49e-01 Dobj:  7.4467935e+01 
Iter:  2 Ap: 1.00e+00 Pobj: -9.5283409e+01 Ad: 5.79e-01 Dobj:  5.2083499e+01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0225091e+02 Ad: 8.53e-01 Dobj:  8.6380288e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0815684e+02 Ad: 9.25e-01 Dobj:  7.0734435e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.1935849e+02 Ad: 8.71e-01 Dobj:  9.0080636e-02 
Iter:  6 Ap: 1.00e+00 Pobj: -1.4328127e+02 Ad: 6.50e-01 Dobj:  5.3469451e-02 
Iter:  7 Ap: 4.67e-01 Pobj: -1.4032953e+02 Ad: 6.14e-01 Dobj:  3.8703033e-02 
Iter:  8 Ap: 4.69e-01 Pobj: -1.3318863e+02 Ad: 5.32e-01 Dobj:  2.6065786e-